In [241]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")

library("car")
require("lattice")
require("ez")

library("cowplot")
library("ggbeeswarm")

library("nortest")

library("effectsize")

theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))


options(width=240)
library(tidyverse)

In [242]:
library(xtable)
library(IRdisplay)
library(repr)
require("papaja")
require("tinytex")

In [243]:
apaMd <- function(apaObj){
    apaStr <- ""
    for (ii in names(apaObj)){
        apaStr <- paste(apaStr, paste("\n","### ", ii, "\n" ,sep="" ,collapse = "")  ,sep="" ,collapse = "")
        if(!(ii == "table")){ 
            for (jj in names(apaObj[[ii]])){
                apaStr <- paste(apaStr, paste("- ", ii," for `", jj, "` is ", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                apaStr <- paste(apaStr, paste("  - ", apaObj[[ii]][[jj]], "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                }}
        else {
            # apaStr <- paste(apaStr, "\n\n", paste(apa_table(apaObj$table))  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- support for will arrive some day", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- for now please use apa_table function", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")

        }}
    return(apaStr)
}

In [244]:
AOV4 <- function(DATA,INFO){

    INFO <- paste("`",INFO,"`",sep="")
    DATA_SUMMARY <- DATA %>%
        group_by(cond0) %>%
        summarise(
            N = n(),
            valX_M = mean(valX),
            valX_SD = sd(valX),
            valX_SE = valX_SD / sqrt(N),
            .groups = "drop"
        )    
    DATA_aov <- aov_4(
        formula  <- FORMULA,
        data     <- DATA,
    )    
    DATA_aov_APA <- apa_print(DATA_aov)
    DATA_aov_EMM <- emmeans(DATA_aov, specs<-c("cond0"))
    DATA_aov_EMM_APA <- apa_print(DATA_aov_EMM)
    DATA_aov_EMM_PAIRS <- pairs(DATA_aov_EMM)
    
    # cont <- "pairwise"
    # DATA_aov_EMM_PAIRS <- contrast(DATA_aov_EMM, cont, by = c("chan0"))
    # DATA_aov_EMM_PAIRS <- contrast(DATA_aov_EMM, cont, by = c("chan0"))    
    
    DATA_aov_EMM_PAIRS_APA <- apa_print(DATA_aov_EMM_PAIRS)    

    return(list("info"      = INFO,
                "summary"   = DATA_SUMMARY,
                "aov"       = DATA_aov,
                "aov_APA"   = DATA_aov_APA,
                "emm"       = DATA_aov_EMM,
                "emm_APA"   = DATA_aov_EMM_APA,
                "pairs"     = DATA_aov_EMM_PAIRS,
                "pairs_APA" = DATA_aov_EMM_PAIRS_APA
               )
          )
}

# Peaks 0

In [155]:
ifName="./data/JOB_004/exports/df3_peaks3_crude.csv"

df0 <- read.csv(file = ifName)
df0 <- subset(df0, CHAN_BUND=="" )
df0 <- subset(df0, quest0=="word_set" )

In [156]:
df1 <- data.frame(df0)

In [157]:
df1 <- subset(df1, cond0 %in% c("noun_F","noun_M","verb_X") )
# df1 <- subset(df1, cond0 %in% c("noun_X","verb_X") )

In [158]:
'%!in%' <- function(x,y)!('%in%'(x,y))

# df1 <- subset(df1, SUB %!in% c("01mkxf","02ppxm","09msxf","15agxf","17makf","24jkxf","26mwxf","31ltxf","33kkxm","37lpxm") )

In [159]:
df1 <- droplevels(df1)
# df1 <- subset(df1, select = -c(evoked0,SES,TASK,RUN,chanX,quest0,CHAN_BUND) )
df1 <- subset(df1, select = c(cond0,chan0,tmin0,tmax0,mode0,valX,SUB) )



In [160]:
str(df1)

'data.frame':	9072 obs. of  7 variables:
 $ cond0: Factor w/ 3 levels "noun_F","noun_M",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ chan0: Factor w/ 6 levels "LC","LF","LP",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ tmin0: num  0.1 0.1 0.19 0.19 0.35 0.35 0.41 0.41 0.6 0.6 ...
 $ tmax0: num  0.19 0.19 0.29 0.29 0.45 0.45 0.58 0.58 0.895 0.895 ...
 $ mode0: Factor w/ 2 levels "neg","pos": 2 1 2 1 2 1 2 1 2 1 ...
 $ valX : num  0.0756 -2.2746 -0.0233 -1.0445 -1.3524 ...
 $ SUB  : Factor w/ 42 levels "01mkxf","02ppxm",..: 1 1 1 1 1 1 1 1 1 1 ...


In [161]:
c(summary(df1["cond0"]))

[1] "noun_F:3024  " "noun_M:3024  " "verb_X:3024  "

In [162]:
c(unique(df1["chan0"]))

$chan0
[1] LF RF LC RC LP RP
Levels: LC LF LP RC RF RP

In [163]:
c(unique(df1["tmin0"]))

$tmin0
[1] 0.10 0.19 0.35 0.41 0.60 0.11

https://www.rdocumentation.org/packages/afex/versions/0.27-2/topics/aov_car

In [164]:
FORMULA  <- valX~(cond0|SUB)
TEST     <- adjusted("free")

In [165]:
CONDS <- c()
mode0 <- as.matrix(unique(df1["mode0"]))
chan0 <- as.matrix(unique(df1["chan0"]))
tmin0 <- as.matrix(unique(df1["tmin0"]))
for(tmin1 in tmin0 ){
    for(chan1 in chan0 ){
        for(mode1 in mode0 ){
            CONDS <- c(CONDS,str_c("tmin0==",tmin1," & chan0==\"",chan1,"\" & mode0==\"",mode1,"\""))
        }
    }
}

In [166]:
length(CONDS)

[1] 72

In [167]:
IDX=0
CONDS[1]

[1] "tmin0==0.1 & chan0==\"LF\" & mode0==\"pos\""

In [168]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 1 of 72

`tmin0==0.1 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.52,1.84,75.42,0.32,.038,.009


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.0798279,1.342364,0.2071313
noun_M,42,0.7825993,1.381289,0.2131375
verb_X,42,0.8373153,1.302212,0.2009357


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.08,"$[0.66$, $1.50]$",5.21,< .001
noun_M,noun_M,0.78,"$[0.37$, $1.20]$",3.78,< .001
verb_X,verb_X,0.84,"$[0.42$, $1.25]$",4.04,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.30,"$[0.01$, $0.58]$",2.49,.039
noun_F_verb_X,noun_F - verb_X,0.24,"$[-0.04$, $0.53]$",2.03,.111
noun_M_verb_X,noun_M - verb_X,-0.05,"$[-0.34$, $0.23]$",-0.46,.891


In [169]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 2 of 72

`tmin0==0.1 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.19,1.82,74.48,0.28,.806,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.775418,1.417731,0.2187607
noun_M,42,-2.750686,1.704491,0.2630087
verb_X,42,-2.708572,1.415776,0.2184591


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.78,"$[-3.25$, $-2.30]$",-11.84,< .001
noun_M,noun_M,-2.75,"$[-3.22$, $-2.28]$",-11.74,< .001
verb_X,verb_X,-2.71,"$[-3.18$, $-2.24]$",-11.56,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.02,"$[-0.29$, $0.24]$",-0.23,.972
noun_F_verb_X,noun_F - verb_X,-0.07,"$[-0.33$, $0.19]$",-0.61,.815
noun_M_verb_X,noun_M - verb_X,-0.04,"$[-0.30$, $0.22]$",-0.38,.922


In [170]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 3 of 72

`tmin0==0.1 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.16,1.85,75.95,0.28,.840,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.3813301,1.277017,0.1970480
noun_M,42,0.3340320,1.107382,0.1708728
verb_X,42,0.3929957,1.149810,0.1774196


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.38,"$[0.02$, $0.75]$",2.09,.041
noun_M,noun_M,0.33,"$[-0.03$, $0.70]$",1.83,.072
verb_X,verb_X,0.39,"$[0.03$, $0.76]$",2.16,.035


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.05,"$[-0.22$, $0.31]$",0.42,.906
noun_F_verb_X,noun_F - verb_X,-0.01,"$[-0.28$, $0.25]$",-0.10,.994
noun_M_verb_X,noun_M - verb_X,-0.06,"$[-0.33$, $0.21]$",-0.53,.858


In [171]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 4 of 72

`tmin0==0.1 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.11,1.76,72.22,0.37,.870,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-3.014649,1.353080,0.2087848
noun_M,42,-3.048192,1.399786,0.2159917
verb_X,42,-2.988902,1.490371,0.2299692


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.01,"$[-3.45$, $-2.58]$",-13.80,< .001
noun_M,noun_M,-3.05,"$[-3.49$, $-2.61]$",-13.96,< .001
verb_X,verb_X,-2.99,"$[-3.43$, $-2.55]$",-13.68,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.03,"$[-0.27$, $0.33]$",0.27,.961
noun_F_verb_X,noun_F - verb_X,-0.03,"$[-0.32$, $0.27]$",-0.21,.977
noun_M_verb_X,noun_M - verb_X,-0.06,"$[-0.36$, $0.24]$",-0.47,.884


In [172]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 5 of 72

`tmin0==0.1 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.09,1.70,69.78,0.22,.139,.007


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.1275563,0.9902205,0.1527943
noun_M,42,0.9490766,0.8701118,0.1342612
verb_X,42,0.9765251,1.0187853,0.1572020


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.13,"$[0.83$, $1.43]$",7.60,< .001
noun_M,noun_M,0.95,"$[0.65$, $1.25]$",6.39,< .001
verb_X,verb_X,0.98,"$[0.68$, $1.27]$",6.58,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.18,"$[-0.05$, $0.40]$",1.90,.146
noun_F_verb_X,noun_F - verb_X,0.15,"$[-0.07$, $0.38]$",1.61,.249
noun_M_verb_X,noun_M - verb_X,-0.03,"$[-0.25$, $0.20]$",-0.29,.954


In [173]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 6 of 72

`tmin0==0.1 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.65,1.94,79.39,0.13,.032,.011


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.029579,0.8369821,0.1291491
noun_M,42,-1.187889,0.8056964,0.1243216
verb_X,42,-0.986402,0.8313299,0.1282770


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.03,"$[-1.28$, $-0.77]$",-8.09,< .001
noun_M,noun_M,-1.19,"$[-1.44$, $-0.93]$",-9.33,< .001
verb_X,verb_X,-0.99,"$[-1.24$, $-0.73]$",-7.75,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.16,"$[-0.03$, $0.35]$",2.02,.115
noun_F_verb_X,noun_F - verb_X,-0.04,"$[-0.23$, $0.14]$",-0.55,.847
noun_M_verb_X,noun_M - verb_X,-0.20,"$[-0.39$, $-0.01]$",-2.57,.032


In [174]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 7 of 72

`tmin0==0.1 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.47,1.91,78.27,0.18,.618,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.109632,0.8754474,0.1350845
noun_M,42,1.031065,0.9490884,0.1464475
verb_X,42,1.105326,1.1565292,0.1784563


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.11,"$[0.80$, $1.42]$",7.19,< .001
noun_M,noun_M,1.03,"$[0.72$, $1.34]$",6.68,< .001
verb_X,verb_X,1.11,"$[0.80$, $1.42]$",7.16,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.08,"$[-0.14$, $0.30]$",0.86,.666
noun_F_verb_X,noun_F - verb_X,0.00,"$[-0.21$, $0.22]$",0.05,.999
noun_M_verb_X,noun_M - verb_X,-0.07,"$[-0.29$, $0.14]$",-0.81,.695


In [175]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 8 of 72

`tmin0==0.1 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.81,1.93,79.08,0.15,.028,.014


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.9116902,0.7653847,0.1181014
noun_M,42,-0.8637751,0.7520701,0.1160469
verb_X,42,-0.6961478,0.8389583,0.1294541


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.91,"$[-1.15$, $-0.67]$",-7.51,< .001
noun_M,noun_M,-0.86,"$[-1.11$, $-0.62]$",-7.12,< .001
verb_X,verb_X,-0.70,"$[-0.94$, $-0.45]$",-5.74,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.05,"$[-0.24$, $0.15]$",-0.58,.829
noun_F_verb_X,noun_F - verb_X,-0.22,"$[-0.41$, $-0.02]$",-2.63,.027
noun_M_verb_X,noun_M - verb_X,-0.17,"$[-0.36$, $0.03]$",-2.04,.108


In [176]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 9 of 72

`tmin0==0.1 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.22,1.66,68.14,0.37,.295,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,4.058145,2.132290,0.3290195
noun_M,42,4.198637,2.032277,0.3135872
verb_X,42,4.017708,2.244133,0.3462773


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,4.06,"$[3.39$, $4.72]$",12.30,< .001
noun_M,noun_M,4.20,"$[3.53$, $4.86]$",12.73,< .001
verb_X,verb_X,4.02,"$[3.35$, $4.68]$",12.18,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.14,"$[-0.43$, $0.15]$",-1.16,.483
noun_F_verb_X,noun_F - verb_X,0.04,"$[-0.25$, $0.33]$",0.33,.941
noun_M_verb_X,noun_M - verb_X,0.18,"$[-0.11$, $0.47]$",1.49,.301


In [177]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 10 of 72

`tmin0==0.1 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.14,1.85,75.91,0.35,.856,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.062577,2.685225,0.4143392
noun_M,42,-1.999838,2.554357,0.3941459
verb_X,42,-2.046829,2.624049,0.4048996


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.06,"$[-2.88$, $-1.25]$",-5.10,< .001
noun_M,noun_M,-2.00,"$[-2.82$, $-1.18]$",-4.94,< .001
verb_X,verb_X,-2.05,"$[-2.86$, $-1.23]$",-5.06,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.06,"$[-0.36$, $0.23]$",-0.51,.869
noun_F_verb_X,noun_F - verb_X,-0.02,"$[-0.31$, $0.28]$",-0.13,.991
noun_M_verb_X,noun_M - verb_X,0.05,"$[-0.25$, $0.34]$",0.38,.924


In [178]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 11 of 72

`tmin0==0.1 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.64,1.81,74.02,0.39,.515,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,5.247797,2.638329,0.4071030
noun_M,42,5.106216,2.669565,0.4119228
verb_X,42,5.210179,2.521894,0.3891366


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,5.25,"$[4.44$, $6.06]$",13.03,< .001
noun_M,noun_M,5.11,"$[4.29$, $5.92]$",12.68,< .001
verb_X,verb_X,5.21,"$[4.40$, $6.02]$",12.93,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.14,"$[-0.17$, $0.45]$",1.09,.522
noun_F_verb_X,noun_F - verb_X,0.04,"$[-0.27$, $0.35]$",0.29,.955
noun_M_verb_X,noun_M - verb_X,-0.10,"$[-0.41$, $0.21]$",-0.80,.703


In [179]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 12 of 72

`tmin0==0.1 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.84,2.00,81.82,0.31,.010,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.6203155,1.791694,0.2764643
noun_M,42,-0.2421227,1.676090,0.2586263
verb_X,42,-0.4475602,1.696056,0.2617071


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.62,"$[-1.15$, $-0.09]$",-2.33,.024
noun_M,noun_M,-0.24,"$[-0.78$, $0.29]$",-0.91,.367
verb_X,verb_X,-0.45,"$[-0.98$, $0.09]$",-1.68,.099


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.38,"$[-0.67$, $-0.09]$",-3.11,.007
noun_F_verb_X,noun_F - verb_X,-0.17,"$[-0.46$, $0.12]$",-1.42,.336
noun_M_verb_X,noun_M - verb_X,0.21,"$[-0.09$, $0.50]$",1.69,.216


In [180]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 13 of 72

`tmin0==0.19 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.85,1.82,74.52,0.33,.167,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.126184,1.796150,0.2771520
noun_M,42,1.954142,1.943283,0.2998551
verb_X,42,1.909379,1.861813,0.2872840


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.13,"$[1.55$, $2.71]$",7.38,< .001
noun_M,noun_M,1.95,"$[1.37$, $2.53]$",6.78,< .001
verb_X,verb_X,1.91,"$[1.33$, $2.49]$",6.62,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.17,"$[-0.11$, $0.46]$",1.45,.322
noun_F_verb_X,noun_F - verb_X,0.22,"$[-0.07$, $0.50]$",1.82,.169
noun_M_verb_X,noun_M - verb_X,0.04,"$[-0.24$, $0.33]$",0.38,.925


In [181]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 14 of 72

`tmin0==0.19 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.14,1.88,77.05,0.40,.855,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.775206,1.772241,0.2734628
noun_M,42,-1.844449,1.824880,0.2815851
verb_X,42,-1.793346,1.680821,0.2593563


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.78,"$[-2.32$, $-1.23]$",-6.54,< .001
noun_M,noun_M,-1.84,"$[-2.39$, $-1.30]$",-6.79,< .001
verb_X,verb_X,-1.79,"$[-2.34$, $-1.25]$",-6.60,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.07,"$[-0.25$, $0.39]$",0.52,.864
noun_F_verb_X,noun_F - verb_X,0.02,"$[-0.30$, $0.34]$",0.14,.990
noun_M_verb_X,noun_M - verb_X,-0.05,"$[-0.37$, $0.27]$",-0.38,.923


In [182]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 15 of 72

`tmin0==0.19 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.67,1.94,79.48,0.28,.031,.005


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.9362706,1.718330,0.2651441
noun_M,42,0.6334266,1.657482,0.2557550
verb_X,42,0.7394403,1.796061,0.2771383


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.94,"$[0.40$, $1.47]$",3.52,.001
noun_M,noun_M,0.63,"$[0.10$, $1.17]$",2.38,.021
verb_X,verb_X,0.74,"$[0.20$, $1.28]$",2.78,.008


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.30,"$[0.03$, $0.57]$",2.67,.024
noun_F_verb_X,noun_F - verb_X,0.20,"$[-0.07$, $0.47]$",1.74,.198
noun_M_verb_X,noun_M - verb_X,-0.11,"$[-0.38$, $0.16]$",-0.93,.620


In [183]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 16 of 72

`tmin0==0.19 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,8.14,1.93,78.98,0.27,.001,.014


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.539765,1.543277,0.2381328
noun_M,42,-2.886127,1.687206,0.2603415
verb_X,42,-2.462653,1.512767,0.2334249


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.54,"$[-3.03$, $-2.05]$",-10.40,< .001
noun_M,noun_M,-2.89,"$[-3.38$, $-2.39]$",-11.82,< .001
verb_X,verb_X,-2.46,"$[-2.95$, $-1.97]$",-10.08,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.35,"$[0.08$, $0.61]$",3.10,.007
noun_F_verb_X,noun_F - verb_X,-0.08,"$[-0.34$, $0.19]$",-0.69,.770
noun_M_verb_X,noun_M - verb_X,-0.42,"$[-0.69$, $-0.16]$",-3.79,.001


In [184]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 17 of 72

`tmin0==0.19 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.13,1.76,72.28,0.18,.855,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.575691,0.9292082,0.1433799
noun_M,42,1.607413,0.9506111,0.1466825
verb_X,42,1.564651,0.9283023,0.1432401


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.58,"$[1.29$, $1.87]$",10.91,< .001
noun_M,noun_M,1.61,"$[1.32$, $1.90]$",11.13,< .001
verb_X,verb_X,1.56,"$[1.27$, $1.85]$",10.83,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.03,"$[-0.24$, $0.18]$",-0.36,.930
noun_F_verb_X,noun_F - verb_X,0.01,"$[-0.20$, $0.22]$",0.13,.991
noun_M_verb_X,noun_M - verb_X,0.04,"$[-0.17$, $0.25]$",0.49,.877


In [185]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 18 of 72

`tmin0==0.19 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.23,1.80,73.93,0.27,.120,.008


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.5085365,1.1987308,0.1849682
noun_M,42,-0.6596478,1.0079953,0.1555371
verb_X,42,-0.4355195,0.9262213,0.1429191


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.51,"$[-0.83$, $-0.18]$",-3.14,.003
noun_M,noun_M,-0.66,"$[-0.98$, $-0.33]$",-4.07,< .001
verb_X,verb_X,-0.44,"$[-0.76$, $-0.11]$",-2.69,.009


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.15,"$[-0.11$, $0.41]$",1.39,.348
noun_F_verb_X,noun_F - verb_X,-0.07,"$[-0.33$, $0.19]$",-0.67,.779
noun_M_verb_X,noun_M - verb_X,-0.22,"$[-0.48$, $0.03]$",-2.07,.103


In [186]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 19 of 72

`tmin0==0.19 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.13,1.86,76.07,0.20,.130,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.445500,1.362452,0.2102309
noun_M,42,1.315690,1.342021,0.2070784
verb_X,42,1.507623,1.499634,0.2313985


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.45,"$[1.01$, $1.88]$",6.68,< .001
noun_M,noun_M,1.32,"$[0.88$, $1.75]$",6.08,< .001
verb_X,verb_X,1.51,"$[1.07$, $1.94]$",6.96,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.13,"$[-0.10$, $0.36]$",1.37,.363
noun_F_verb_X,noun_F - verb_X,-0.06,"$[-0.29$, $0.16]$",-0.65,.790
noun_M_verb_X,noun_M - verb_X,-0.19,"$[-0.42$, $0.03]$",-2.02,.113


In [187]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 20 of 72

`tmin0==0.19 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.77,1.80,73.82,0.22,.074,.007


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.7314090,0.9265941,0.1429766
noun_M,42,-0.8596825,1.2248853,0.1890039
verb_X,42,-0.6296953,1.1525894,0.1778484


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.73,"$[-1.07$, $-0.39]$",-4.28,< .001
noun_M,noun_M,-0.86,"$[-1.20$, $-0.52]$",-5.03,< .001
verb_X,verb_X,-0.63,"$[-0.97$, $-0.29]$",-3.68,.001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.13,"$[-0.11$, $0.36]$",1.31,.393
noun_F_verb_X,noun_F - verb_X,-0.10,"$[-0.34$, $0.13]$",-1.04,.554
noun_M_verb_X,noun_M - verb_X,-0.23,"$[-0.46$, $0.00]$",-2.35,.055


In [188]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 21 of 72

`tmin0==0.19 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,9.74,1.94,79.70,0.29,< .001,.012


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.954209,1.984943,0.3062834
noun_M,42,2.361286,1.922784,0.2966921
verb_X,42,1.886678,1.968725,0.3037809


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.95,"$[1.35$, $2.56]$",6.46,< .001
noun_M,noun_M,2.36,"$[1.75$, $2.97]$",7.81,< .001
verb_X,verb_X,1.89,"$[1.28$, $2.50]$",6.24,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.41,"$[-0.68$, $-0.13]$",-3.50,.002
noun_F_verb_X,noun_F - verb_X,0.07,"$[-0.21$, $0.35]$",0.58,.831
noun_M_verb_X,noun_M - verb_X,0.47,"$[0.20$, $0.75]$",4.08,< .001


In [189]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 22 of 72

`tmin0==0.19 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.63,1.74,71.45,0.42,.037,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.234740,2.625006,0.4050472
noun_M,42,-1.884800,2.539065,0.3917863
verb_X,42,-2.104852,2.629377,0.4057216


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.23,"$[-3.04$, $-1.43]$",-5.57,< .001
noun_M,noun_M,-1.88,"$[-2.69$, $-1.08]$",-4.70,< .001
verb_X,verb_X,-2.10,"$[-2.91$, $-1.30]$",-5.25,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.35,"$[-0.66$, $-0.04]$",-2.67,.025
noun_F_verb_X,noun_F - verb_X,-0.13,"$[-0.44$, $0.18]$",-0.99,.585
noun_M_verb_X,noun_M - verb_X,0.22,"$[-0.09$, $0.53]$",1.68,.220


In [190]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 23 of 72

`tmin0==0.19 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.61,1.99,81.49,0.38,.206,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,3.747019,2.667446,0.4115958
noun_M,42,3.908017,2.873509,0.4433920
verb_X,42,3.674008,2.548696,0.3932724


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,3.75,"$[2.91$, $4.59]$",8.99,< .001
noun_M,noun_M,3.91,"$[3.07$, $4.75]$",9.38,< .001
verb_X,verb_X,3.67,"$[2.83$, $4.51]$",8.82,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.16,"$[-0.48$, $0.16]$",-1.21,.452
noun_F_verb_X,noun_F - verb_X,0.07,"$[-0.25$, $0.39]$",0.55,.848
noun_M_verb_X,noun_M - verb_X,0.23,"$[-0.08$, $0.55]$",1.75,.191


In [191]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 24 of 72

`tmin0==0.19 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.88,1.81,74.21,0.33,.164,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.6207037,2.270806,0.3503930
noun_M,42,-0.4108307,2.318604,0.3577683
verb_X,42,-0.4336474,2.239387,0.3455448


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.62,"$[-1.33$, $0.09]$",-1.77,.084
noun_M,noun_M,-0.41,"$[-1.12$, $0.30]$",-1.17,.248
verb_X,verb_X,-0.43,"$[-1.14$, $0.27]$",-1.23,.224


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.21,"$[-0.49$, $0.07]$",-1.77,.187
noun_F_verb_X,noun_F - verb_X,-0.19,"$[-0.47$, $0.10]$",-1.57,.262
noun_M_verb_X,noun_M - verb_X,0.02,"$[-0.26$, $0.31]$",0.19,.980


In [192]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 25 of 72

`tmin0==0.35 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.99,1.83,75.17,0.57,.147,.006


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.006300,1.770892,0.2732545
noun_M,42,-1.051203,1.712282,0.2642109
verb_X,42,-1.299351,1.519235,0.2344230


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.01,"$[-1.52$, $-0.49]$",-3.90,< .001
noun_M,noun_M,-1.05,"$[-1.57$, $-0.53]$",-4.08,< .001
verb_X,verb_X,-1.30,"$[-1.82$, $-0.78]$",-5.04,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.04,"$[-0.33$, $0.42]$",0.28,.957
noun_F_verb_X,noun_F - verb_X,0.29,"$[-0.08$, $0.67]$",1.85,.159
noun_M_verb_X,noun_M - verb_X,0.25,"$[-0.13$, $0.63]$",1.57,.265


In [193]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 26 of 72

`tmin0==0.35 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.95,1.83,74.95,0.50,.383,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-3.181257,1.794249,0.2768587
noun_M,42,-3.093118,1.569629,0.2421990
verb_X,42,-2.977539,1.606059,0.2478203


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.18,"$[-3.70$, $-2.67]$",-12.42,< .001
noun_M,noun_M,-3.09,"$[-3.61$, $-2.58]$",-12.08,< .001
verb_X,verb_X,-2.98,"$[-3.49$, $-2.46]$",-11.63,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.09,"$[-0.44$, $0.26]$",-0.60,.823
noun_F_verb_X,noun_F - verb_X,-0.20,"$[-0.56$, $0.15]$",-1.38,.357
noun_M_verb_X,noun_M - verb_X,-0.12,"$[-0.47$, $0.24]$",-0.78,.715


In [194]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 27 of 72

`tmin0==0.35 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.49,1.90,78.07,0.52,.038,.010


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.562104,1.691790,0.2610488
noun_M,42,-1.968772,1.582795,0.2442305
verb_X,42,-1.750961,1.719795,0.2653702


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.56,"$[-2.08$, $-1.05]$",-6.08,< .001
noun_M,noun_M,-1.97,"$[-2.48$, $-1.45]$",-7.66,< .001
verb_X,verb_X,-1.75,"$[-2.27$, $-1.24]$",-6.81,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.41,"$[0.04$, $0.77]$",2.64,.027
noun_F_verb_X,noun_F - verb_X,0.19,"$[-0.18$, $0.56]$",1.23,.442
noun_M_verb_X,noun_M - verb_X,-0.22,"$[-0.59$, $0.15]$",-1.41,.339


In [195]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 28 of 72

`tmin0==0.35 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,5.30,1.90,78.07,0.35,.008,.010


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-3.680973,1.738828,0.2683070
noun_M,42,-3.840452,1.635038,0.2522918
verb_X,42,-3.433158,1.792537,0.2765945


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.68,"$[-4.22$, $-3.15]$",-13.84,< .001
noun_M,noun_M,-3.84,"$[-4.38$, $-3.31]$",-14.44,< .001
verb_X,verb_X,-3.43,"$[-3.97$, $-2.90]$",-12.91,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.16,"$[-0.14$, $0.46]$",1.27,.419
noun_F_verb_X,noun_F - verb_X,-0.25,"$[-0.55$, $0.05]$",-1.97,.127
noun_M_verb_X,noun_M - verb_X,-0.41,"$[-0.71$, $-0.11]$",-3.23,.005


In [196]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 29 of 72

`tmin0==0.35 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.27,1.99,81.58,0.22,.763,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.659264,1.349710,0.2082648
noun_M,42,1.641222,1.254165,0.1935219
verb_X,42,1.586564,1.232751,0.1902176


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.66,"$[1.26$, $2.06]$",8.40,< .001
noun_M,noun_M,1.64,"$[1.24$, $2.04]$",8.31,< .001
verb_X,verb_X,1.59,"$[1.19$, $1.98]$",8.03,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.02,"$[-0.23$, $0.26]$",0.17,.983
noun_F_verb_X,noun_F - verb_X,0.07,"$[-0.17$, $0.32]$",0.70,.761
noun_M_verb_X,noun_M - verb_X,0.05,"$[-0.19$, $0.30]$",0.53,.857


In [197]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 30 of 72

`tmin0==0.35 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.49,1.87,76.56,0.24,.232,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.2299094,1.297205,0.2001631
noun_M,42,0.3421153,1.224453,0.1889373
verb_X,42,0.4056508,1.265236,0.1952302


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.23,"$[-0.16$, $0.62]$",1.18,.244
noun_M,noun_M,0.34,"$[-0.05$, $0.73]$",1.76,.085
verb_X,verb_X,0.41,"$[0.01$, $0.80]$",2.08,.043


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.11,"$[-0.36$, $0.13]$",-1.09,.523
noun_F_verb_X,noun_F - verb_X,-0.18,"$[-0.42$, $0.07]$",-1.71,.209
noun_M_verb_X,noun_M - verb_X,-0.06,"$[-0.31$, $0.18]$",-0.62,.811


In [198]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 31 of 72

`tmin0==0.35 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.10,1.96,80.32,0.22,.130,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.801751,1.514716,0.2337258
noun_M,42,1.631761,1.335056,0.2060036
verb_X,42,1.819825,1.542495,0.2380122


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.80,"$[1.35$, $2.26]$",7.96,< .001
noun_M,noun_M,1.63,"$[1.18$, $2.09]$",7.21,< .001
verb_X,verb_X,1.82,"$[1.36$, $2.28]$",8.04,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.17,"$[-0.07$, $0.41]$",1.68,.220
noun_F_verb_X,noun_F - verb_X,-0.02,"$[-0.26$, $0.22]$",-0.18,.983
noun_M_verb_X,noun_M - verb_X,-0.19,"$[-0.43$, $0.05]$",-1.86,.158


In [199]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 32 of 72

`tmin0==0.35 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,5.24,1.74,71.54,0.33,.010,.011


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.3926471,1.540043,0.2376337
noun_M,42,0.1830027,1.497887,0.2311290
verb_X,42,0.5632380,1.512045,0.2333137


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.39,"$[-0.08$, $0.86]$",1.68,.100
noun_M,noun_M,0.18,"$[-0.29$, $0.65]$",0.78,.438
verb_X,verb_X,0.56,"$[0.09$, $1.03]$",2.41,.020


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.21,"$[-0.07$, $0.49]$",1.78,.182
noun_F_verb_X,noun_F - verb_X,-0.17,"$[-0.45$, $0.11]$",-1.45,.321
noun_M_verb_X,noun_M - verb_X,-0.38,"$[-0.66$, $-0.10]$",-3.23,.005


In [200]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 33 of 72

`tmin0==0.35 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.61,1.82,74.63,0.45,.208,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,3.018245,2.458999,0.3794318
noun_M,42,3.179876,2.032700,0.3136524
verb_X,42,2.933987,2.314439,0.3571257


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,3.02,"$[2.31$, $3.73]$",8.60,< .001
noun_M,noun_M,3.18,"$[2.47$, $3.89]$",9.06,< .001
verb_X,verb_X,2.93,"$[2.23$, $3.64]$",8.36,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.16,"$[-0.49$, $0.17]$",-1.16,.479
noun_F_verb_X,noun_F - verb_X,0.08,"$[-0.25$, $0.42]$",0.61,.818
noun_M_verb_X,noun_M - verb_X,0.25,"$[-0.09$, $0.58]$",1.77,.187


In [201]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 34 of 72

`tmin0==0.35 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.25,1.81,74.22,0.59,.021,.008


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.8547186,2.369555,0.3656303
noun_M,42,1.2097084,1.972308,0.3043338
verb_X,42,1.2902751,2.187197,0.3374919


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.85,"$[0.18$, $1.53]$",2.54,.014
noun_M,noun_M,1.21,"$[0.53$, $1.89]$",3.59,.001
verb_X,verb_X,1.29,"$[0.61$, $1.97]$",3.83,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.35,"$[-0.73$, $0.02]$",-2.23,.072
noun_F_verb_X,noun_F - verb_X,-0.44,"$[-0.82$, $-0.06]$",-2.74,.020
noun_M_verb_X,noun_M - verb_X,-0.08,"$[-0.46$, $0.30]$",-0.51,.868


In [202]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 35 of 72

`tmin0==0.35 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.05,1.88,76.91,0.39,.138,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,4.405957,2.190354,0.3379789
noun_M,42,4.536946,2.030795,0.3133585
verb_X,42,4.268821,2.156623,0.3327742


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,4.41,"$[3.75$, $5.07]$",13.42,< .001
noun_M,noun_M,4.54,"$[3.88$, $5.20]$",13.82,< .001
verb_X,verb_X,4.27,"$[3.61$, $4.93]$",13.01,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.13,"$[-0.45$, $0.19]$",-0.99,.586
noun_F_verb_X,noun_F - verb_X,0.14,"$[-0.18$, $0.45]$",1.04,.557
noun_M_verb_X,noun_M - verb_X,0.27,"$[-0.05$, $0.58]$",2.03,.113


In [203]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 36 of 72

`tmin0==0.35 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.26,1.83,74.94,0.54,.288,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.059869,2.209585,0.3409464
noun_M,42,2.129773,2.343606,0.3616263
verb_X,42,2.296084,2.168927,0.3346727


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.06,"$[1.36$, $2.76]$",5.95,< .001
noun_M,noun_M,2.13,"$[1.43$, $2.83]$",6.16,< .001
verb_X,verb_X,2.30,"$[1.60$, $2.99]$",6.64,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.07,"$[-0.43$, $0.30]$",-0.46,.891
noun_F_verb_X,noun_F - verb_X,-0.24,"$[-0.60$, $0.13]$",-1.54,.276
noun_M_verb_X,noun_M - verb_X,-0.17,"$[-0.53$, $0.20]$",-1.09,.524


In [204]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 37 of 72

`tmin0==0.41 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.74,1.83,74.88,0.67,.469,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.8189044,2.354618,0.3633254
noun_M,42,-0.9276137,1.894097,0.2922655
verb_X,42,-1.0263146,2.128044,0.3283643


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.82,"$[-1.48$, $-0.16]$",-2.49,.016
noun_M,noun_M,-0.93,"$[-1.59$, $-0.27]$",-2.82,.007
verb_X,verb_X,-1.03,"$[-1.69$, $-0.36]$",-3.12,.003


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.11,"$[-0.30$, $0.52]$",0.64,.800
noun_F_verb_X,noun_F - verb_X,0.21,"$[-0.20$, $0.61]$",1.22,.447
noun_M_verb_X,noun_M - verb_X,0.10,"$[-0.31$, $0.51]$",0.58,.832


In [205]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 38 of 72

`tmin0==0.41 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.79,1.83,75.12,0.49,.177,.004


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-3.173741,1.883376,0.2906112
noun_M,42,-3.381702,1.832974,0.2828340
verb_X,42,-3.118111,1.650071,0.2546115


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.17,"$[-3.73$, $-2.62]$",-11.48,< .001
noun_M,noun_M,-3.38,"$[-3.94$, $-2.83]$",-12.23,< .001
verb_X,verb_X,-3.12,"$[-3.67$, $-2.56]$",-11.28,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.21,"$[-0.14$, $0.56]$",1.42,.337
noun_F_verb_X,noun_F - verb_X,-0.06,"$[-0.41$, $0.29]$",-0.38,.924
noun_M_verb_X,noun_M - verb_X,-0.26,"$[-0.61$, $0.09]$",-1.80,.178


In [206]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 39 of 72

`tmin0==0.41 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.67,1.34,55.12,0.96,.097,.005


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.9144348,2.608244,0.4024607
noun_M,42,-1.3031473,1.816353,0.2802694
verb_X,42,-1.0133290,2.581443,0.3983253


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.91,"$[-1.65$, $-0.18]$",-2.51,.016
noun_M,noun_M,-1.30,"$[-2.04$, $-0.57]$",-3.57,.001
verb_X,verb_X,-1.01,"$[-1.75$, $-0.28]$",-2.78,.008


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.39,"$[-0.03$, $0.81]$",2.22,.073
noun_F_verb_X,noun_F - verb_X,0.10,"$[-0.32$, $0.52]$",0.57,.839
noun_M_verb_X,noun_M - verb_X,-0.29,"$[-0.71$, $0.13]$",-1.66,.228


In [207]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 40 of 72

`tmin0==0.41 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,5.06,1.77,72.54,0.38,.011,.008


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-3.546438,1.781913,0.2749552
noun_M,42,-3.812141,1.786266,0.2756268
verb_X,42,-3.417842,1.930124,0.2978245


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.55,"$[-4.12$, $-2.98]$",-12.53,< .001
noun_M,noun_M,-3.81,"$[-4.38$, $-3.24]$",-13.47,< .001
verb_X,verb_X,-3.42,"$[-3.99$, $-2.85]$",-12.08,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.27,"$[-0.04$, $0.57]$",2.10,.096
noun_F_verb_X,noun_F - verb_X,-0.13,"$[-0.43$, $0.17]$",-1.02,.568
noun_M_verb_X,noun_M - verb_X,-0.39,"$[-0.70$, $-0.09]$",-3.12,.007


In [208]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 41 of 72

`tmin0==0.41 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.49,1.74,71.54,0.28,.587,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.620320,1.334331,0.2058917
noun_M,42,1.713846,1.208844,0.1865286
verb_X,42,1.623066,1.231181,0.1899753


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.62,"$[1.23$, $2.01]$",8.34,< .001
noun_M,noun_M,1.71,"$[1.32$, $2.10]$",8.82,< .001
verb_X,verb_X,1.62,"$[1.23$, $2.01]$",8.35,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.09,"$[-0.35$, $0.16]$",-0.87,.659
noun_F_verb_X,noun_F - verb_X,0.00,"$[-0.26$, $0.25]$",-0.03,> .999
noun_M_verb_X,noun_M - verb_X,0.09,"$[-0.17$, $0.35]$",0.85,.675


In [209]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 42 of 72

`tmin0==0.41 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.28,1.65,67.50,0.31,.280,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.06432624,1.534239,0.2367382
noun_M,42,0.07653374,1.283750,0.1980869
verb_X,42,0.22208921,1.550286,0.2392144


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.06,"$[-0.39$, $0.52]$",0.29,.777
noun_M,noun_M,0.08,"$[-0.38$, $0.53]$",0.34,.736
verb_X,verb_X,0.22,"$[-0.23$, $0.68]$",0.99,.330


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.01,"$[-0.27$, $0.25]$",-0.11,.993
noun_F_verb_X,noun_F - verb_X,-0.16,"$[-0.42$, $0.10]$",-1.44,.326
noun_M_verb_X,noun_M - verb_X,-0.15,"$[-0.41$, $0.12]$",-1.33,.384


In [210]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 43 of 72

`tmin0==0.41 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.22,1.73,70.89,0.36,.771,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.886977,1.413982,0.2181822
noun_M,42,1.808451,1.426966,0.2201856
verb_X,42,1.830437,1.507021,0.2325384


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.89,"$[1.44$, $2.34]$",8.43,< .001
noun_M,noun_M,1.81,"$[1.36$, $2.26]$",8.08,< .001
verb_X,verb_X,1.83,"$[1.38$, $2.28]$",8.18,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.08,"$[-0.21$, $0.37]$",0.64,.797
noun_F_verb_X,noun_F - verb_X,0.06,"$[-0.23$, $0.35]$",0.46,.889
noun_M_verb_X,noun_M - verb_X,-0.02,"$[-0.31$, $0.27]$",-0.18,.982


In [211]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 44 of 72

`tmin0==0.41 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.87,1.60,65.71,0.42,.034,.009


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.2531982,1.603689,0.2474545
noun_M,42,0.1365818,1.487606,0.2295425
verb_X,42,0.4823398,1.547356,0.2387622


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.25,"$[-0.23$, $0.73]$",1.06,.294
noun_M,noun_M,0.14,"$[-0.34$, $0.62]$",0.57,.570
verb_X,verb_X,0.48,"$[0.00$, $0.96]$",2.02,.049


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.12,"$[-0.19$, $0.42]$",0.92,.628
noun_F_verb_X,noun_F - verb_X,-0.23,"$[-0.53$, $0.07]$",-1.81,.172
noun_M_verb_X,noun_M - verb_X,-0.35,"$[-0.65$, $-0.04]$",-2.73,.021


In [212]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 45 of 72

`tmin0==0.41 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.84,1.99,81.43,0.41,.065,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.623229,2.453761,0.3786235
noun_M,42,2.934625,2.145465,0.3310524
verb_X,42,2.677878,2.356399,0.3636002


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.62,"$[1.90$, $3.34]$",7.32,< .001
noun_M,noun_M,2.93,"$[2.21$, $3.66]$",8.19,< .001
verb_X,verb_X,2.68,"$[1.96$, $3.40]$",7.47,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.31,"$[-0.64$, $0.02]$",-2.23,.072
noun_F_verb_X,noun_F - verb_X,-0.05,"$[-0.39$, $0.28]$",-0.39,.919
noun_M_verb_X,noun_M - verb_X,0.26,"$[-0.08$, $0.59]$",1.84,.163


In [213]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 46 of 72

`tmin0==0.41 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.86,1.41,57.91,1.00,.082,.005


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.22539826,2.868871,0.4426764
noun_M,42,0.21103173,2.158873,0.3331213
verb_X,42,0.03857912,2.928277,0.4518430


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.23,"$[-1.06$, $0.61]$",-0.55,.588
noun_M,noun_M,0.21,"$[-0.62$, $1.04]$",0.51,.612
verb_X,verb_X,0.04,"$[-0.79$, $0.87]$",0.09,.926


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.44,"$[-0.87$, $0.00]$",-2.38,.051
noun_F_verb_X,noun_F - verb_X,-0.26,"$[-0.70$, $0.17]$",-1.44,.327
noun_M_verb_X,noun_M - verb_X,0.17,"$[-0.27$, $0.61]$",0.94,.617


In [214]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 47 of 72

`tmin0==0.41 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.94,1.83,75.05,0.40,.063,.004


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,3.984285,2.270478,0.3503423
noun_M,42,4.279039,2.114851,0.3263285
verb_X,42,4.025614,2.171331,0.3350437


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,3.98,"$[3.30$, $4.66]$",11.81,< .001
noun_M,noun_M,4.28,"$[3.60$, $4.96]$",12.68,< .001
verb_X,verb_X,4.03,"$[3.35$, $4.70]$",11.93,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.29,"$[-0.61$, $0.02]$",-2.24,.070
noun_F_verb_X,noun_F - verb_X,-0.04,"$[-0.36$, $0.27]$",-0.31,.947
noun_M_verb_X,noun_M - verb_X,0.25,"$[-0.06$, $0.57]$",1.93,.138


In [215]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 48 of 72

`tmin0==0.41 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.46,1.61,65.90,0.84,.591,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.217192,2.954751,0.4559280
noun_M,42,1.350855,2.488907,0.3840466
verb_X,42,1.377196,2.894959,0.4467019


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.22,"$[0.35$, $2.08]$",2.83,.007
noun_M,noun_M,1.35,"$[0.49$, $2.22]$",3.14,.003
verb_X,verb_X,1.38,"$[0.51$, $2.24]$",3.20,.002


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.13,"$[-0.56$, $0.29]$",-0.75,.736
noun_F_verb_X,noun_F - verb_X,-0.16,"$[-0.59$, $0.27]$",-0.89,.646
noun_M_verb_X,noun_M - verb_X,-0.03,"$[-0.45$, $0.40]$",-0.15,.988


In [216]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 49 of 72

`tmin0==0.6 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,6.08,1.70,69.68,0.76,.006,.013


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.07994334,2.487650,0.3838527
noun_M,42,-0.30085929,2.100230,0.3240725
verb_X,42,-0.52473504,1.996413,0.3080532


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.08,"$[-0.60$, $0.76]$",0.23,.815
noun_M,noun_M,-0.30,"$[-0.98$, $0.38]$",-0.88,.381
verb_X,verb_X,-0.52,"$[-1.21$, $0.16]$",-1.54,.129


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.38,"$[-0.04$, $0.80]$",2.17,.082
noun_F_verb_X,noun_F - verb_X,0.60,"$[0.19$, $1.02]$",3.45,.003
noun_M_verb_X,noun_M - verb_X,0.22,"$[-0.19$, $0.64]$",1.28,.413


In [217]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 50 of 72

`tmin0==0.6 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.53,1.99,81.56,0.45,.592,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.467235,1.766788,0.2726213
noun_M,42,-2.584253,1.828919,0.2822083
verb_X,42,-2.444123,1.585923,0.2447133


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.47,"$[-3.00$, $-1.93]$",-9.24,< .001
noun_M,noun_M,-2.58,"$[-3.12$, $-2.05]$",-9.68,< .001
verb_X,verb_X,-2.44,"$[-2.98$, $-1.91]$",-9.15,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.12,"$[-0.23$, $0.47]$",0.80,.705
noun_F_verb_X,noun_F - verb_X,-0.02,"$[-0.37$, $0.33]$",-0.16,.986
noun_M_verb_X,noun_M - verb_X,-0.14,"$[-0.49$, $0.21]$",-0.96,.607


In [218]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 51 of 72

`tmin0==0.6 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,7.13,1.49,61.00,0.66,.004,.012


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.757297,2.494059,0.3848417
noun_M,42,1.226202,1.951343,0.3010987
verb_X,42,1.291607,2.141923,0.3305059


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.76,"$[1.07$, $2.44]$",5.16,< .001
noun_M,noun_M,1.23,"$[0.54$, $1.91]$",3.60,.001
verb_X,verb_X,1.29,"$[0.61$, $1.98]$",3.79,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.53,"$[0.17$, $0.90]$",3.46,.002
noun_F_verb_X,noun_F - verb_X,0.47,"$[0.10$, $0.83]$",3.04,.009
noun_M_verb_X,noun_M - verb_X,-0.07,"$[-0.43$, $0.30]$",-0.43,.905


In [219]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 52 of 72

`tmin0==0.6 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.97,1.96,80.40,0.39,.010,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.537796,1.946265,0.3003152
noun_M,42,-1.786438,1.917218,0.2958331
verb_X,42,-1.363889,1.896626,0.2926557


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.54,"$[-2.13$, $-0.94]$",-5.19,< .001
noun_M,noun_M,-1.79,"$[-2.38$, $-1.19]$",-6.03,< .001
verb_X,verb_X,-1.36,"$[-1.96$, $-0.77]$",-4.60,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.25,"$[-0.07$, $0.57]$",1.85,.161
noun_F_verb_X,noun_F - verb_X,-0.17,"$[-0.50$, $0.15]$",-1.29,.404
noun_M_verb_X,noun_M - verb_X,-0.42,"$[-0.74$, $-0.10]$",-3.14,.007


In [220]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 53 of 72

`tmin0==0.6 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.77,1.88,77.12,0.20,.179,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.1520551,1.298960,0.2004338
noun_M,42,1.0560081,1.288715,0.1988531
verb_X,42,0.9724347,1.340076,0.2067782


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.15,"$[0.75$, $1.56]$",5.70,< .001
noun_M,noun_M,1.06,"$[0.65$, $1.46]$",5.23,< .001
verb_X,verb_X,0.97,"$[0.57$, $1.38]$",4.81,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.10,"$[-0.13$, $0.32]$",1.00,.576
noun_F_verb_X,noun_F - verb_X,0.18,"$[-0.05$, $0.41]$",1.88,.151
noun_M_verb_X,noun_M - verb_X,0.08,"$[-0.14$, $0.31]$",0.87,.658


In [221]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 54 of 72

`tmin0==0.6 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.13,1.78,72.88,0.32,.055,.007


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.7694636,1.521504,0.2347731
noun_M,42,-0.7500802,1.357428,0.2094558
verb_X,42,-0.5079675,1.529783,0.2360506


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.77,"$[-1.23$, $-0.31]$",-3.39,.001
noun_M,noun_M,-0.75,"$[-1.21$, $-0.29]$",-3.30,.002
verb_X,verb_X,-0.51,"$[-0.96$, $-0.05]$",-2.24,.030


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.02,"$[-0.30$, $0.26]$",-0.17,.985
noun_F_verb_X,noun_F - verb_X,-0.26,"$[-0.54$, $0.02]$",-2.25,.070
noun_M_verb_X,noun_M - verb_X,-0.24,"$[-0.52$, $0.04]$",-2.08,.100


In [222]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 55 of 72

`tmin0==0.6 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.76,1.76,72.29,0.39,.183,.006


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.290537,1.255794,0.1937732
noun_M,42,2.048997,1.308174,0.2018557
verb_X,42,2.164293,1.360651,0.2099530


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.29,"$[1.89$, $2.70]$",11.34,< .001
noun_M,noun_M,2.05,"$[1.64$, $2.45]$",10.15,< .001
verb_X,verb_X,2.16,"$[1.76$, $2.57]$",10.72,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.24,"$[-0.07$, $0.55]$",1.88,.152
noun_F_verb_X,noun_F - verb_X,0.13,"$[-0.18$, $0.43]$",0.98,.591
noun_M_verb_X,noun_M - verb_X,-0.12,"$[-0.42$, $0.19]$",-0.90,.644


In [223]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 56 of 72

`tmin0==0.6 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,9.23,1.73,70.75,0.42,.001,.024


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.4552257,1.543887,0.2382269
noun_M,42,0.2476184,1.416440,0.2185614
verb_X,42,0.8087916,1.479566,0.2283020


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.46,"$[0.00$, $0.91]$",1.99,.052
noun_M,noun_M,0.25,"$[-0.21$, $0.71]$",1.08,.284
verb_X,verb_X,0.81,"$[0.35$, $1.27]$",3.54,.001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.21,"$[-0.11$, $0.52]$",1.57,.264
noun_F_verb_X,noun_F - verb_X,-0.35,"$[-0.67$, $-0.04]$",-2.68,.024
noun_M_verb_X,noun_M - verb_X,-0.56,"$[-0.88$, $-0.25]$",-4.25,< .001


In [224]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 57 of 72

`tmin0==0.6 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.54,1.91,78.34,0.46,.576,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.7134675,2.140620,0.3303049
noun_M,42,0.8036971,2.189929,0.3379133
verb_X,42,0.6535181,2.113999,0.3261971


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.71,"$[0.05$, $1.38]$",2.15,.037
noun_M,noun_M,0.80,"$[0.14$, $1.47]$",2.42,.019
verb_X,verb_X,0.65,"$[-0.01$, $1.32]$",1.97,.055


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.09,"$[-0.44$, $0.26]$",-0.62,.809
noun_F_verb_X,noun_F - verb_X,0.06,"$[-0.29$, $0.41]$",0.41,.911
noun_M_verb_X,noun_M - verb_X,0.15,"$[-0.20$, $0.50]$",1.03,.558


In [225]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 58 of 72

`tmin0==0.6 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,10.14,1.73,70.95,0.67,< .001,.016


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.252096,2.565188,0.3958171
noun_M,42,-1.746928,2.213455,0.3415435
verb_X,42,-1.519543,2.490908,0.3843555


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.25,"$[-3.01$, $-1.50]$",-6.01,< .001
noun_M,noun_M,-1.75,"$[-2.50$, $-0.99]$",-4.66,< .001
verb_X,verb_X,-1.52,"$[-2.27$, $-0.77]$",-4.06,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.51,"$[-0.90$, $-0.11]$",-3.03,.009
noun_F_verb_X,noun_F - verb_X,-0.73,"$[-1.13$, $-0.34]$",-4.40,< .001
noun_M_verb_X,noun_M - verb_X,-0.23,"$[-0.62$, $0.17]$",-1.37,.364


In [226]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 59 of 72

`tmin0==0.6 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.75,1.92,78.72,0.47,.181,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.367871,2.104310,0.3247021
noun_M,42,2.623882,2.232243,0.3444426
verb_X,42,2.408902,1.933264,0.2983091


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.37,"$[1.72$, $3.02]$",7.33,< .001
noun_M,noun_M,2.62,"$[1.97$, $3.27]$",8.12,< .001
verb_X,verb_X,2.41,"$[1.76$, $3.06]$",7.46,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.26,"$[-0.61$, $0.09]$",-1.74,.195
noun_F_verb_X,noun_F - verb_X,-0.04,"$[-0.39$, $0.31]$",-0.28,.958
noun_M_verb_X,noun_M - verb_X,0.21,"$[-0.14$, $0.57]$",1.46,.313


In [227]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 60 of 72

`tmin0==0.6 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,6.13,1.69,69.24,0.80,.006,.009


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.2944131,3.049104,0.4704870
noun_M,42,0.1770048,2.502737,0.3861808
verb_X,42,0.2981776,2.547928,0.3931539


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.29,"$[-1.14$, $0.55]$",-0.70,.485
noun_M,noun_M,0.18,"$[-0.66$, $1.02]$",0.42,.674
verb_X,verb_X,0.30,"$[-0.54$, $1.14]$",0.71,.480


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.47,"$[-0.90$, $-0.04]$",-2.64,.027
noun_F_verb_X,noun_F - verb_X,-0.59,"$[-1.02$, $-0.17]$",-3.31,.004
noun_M_verb_X,noun_M - verb_X,-0.12,"$[-0.55$, $0.31]$",-0.68,.777


In [228]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 61 of 72

`tmin0==0.11 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.56,1.75,71.85,0.33,.039,.011


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.7275727,1.416942,0.2186389
noun_M,42,-1.0345865,1.194638,0.1843367
verb_X,42,-0.9408438,1.062997,0.1640240


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.73,"$[-1.11$, $-0.35]$",-3.82,< .001
noun_M,noun_M,-1.03,"$[-1.42$, $-0.65]$",-5.44,< .001
verb_X,verb_X,-0.94,"$[-1.32$, $-0.56]$",-4.94,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.31,"$[0.03$, $0.59]$",2.60,.029
noun_F_verb_X,noun_F - verb_X,0.21,"$[-0.07$, $0.49]$",1.81,.173
noun_M_verb_X,noun_M - verb_X,-0.09,"$[-0.38$, $0.19]$",-0.79,.707


In [229]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 62 of 72

`tmin0==0.11 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.03,1.64,67.17,0.38,.944,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.329009,1.555660,0.2400436
noun_M,42,-2.328074,1.510483,0.2330727
verb_X,42,-2.356017,1.411975,0.2178725


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.33,"$[-2.79$, $-1.87]$",-10.10,< .001
noun_M,noun_M,-2.33,"$[-2.79$, $-1.86]$",-10.10,< .001
verb_X,verb_X,-2.36,"$[-2.82$, $-1.89]$",-10.22,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.00,"$[-0.29$, $0.29]$",-0.01,> .999
noun_F_verb_X,noun_F - verb_X,0.03,"$[-0.26$, $0.32]$",0.22,.973
noun_M_verb_X,noun_M - verb_X,0.03,"$[-0.26$, $0.32]$",0.23,.972


In [230]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 63 of 72

`tmin0==0.11 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.32,1.86,76.39,0.25,.709,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.109571,1.217147,0.1878098
noun_M,42,-1.146987,1.095491,0.1690379
verb_X,42,-1.062058,1.110523,0.1713574


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.11,"$[-1.46$, $-0.76]$",-6.29,< .001
noun_M,noun_M,-1.15,"$[-1.50$, $-0.79]$",-6.51,< .001
verb_X,verb_X,-1.06,"$[-1.42$, $-0.71]$",-6.03,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.04,"$[-0.22$, $0.29]$",0.35,.933
noun_F_verb_X,noun_F - verb_X,-0.05,"$[-0.30$, $0.21]$",-0.45,.895
noun_M_verb_X,noun_M - verb_X,-0.08,"$[-0.34$, $0.17]$",-0.80,.703


In [231]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 64 of 72

`tmin0==0.11 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.24,1.74,71.30,0.33,.759,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.393030,1.423830,0.2197018
noun_M,42,-2.452465,1.398114,0.2157337
verb_X,42,-2.376120,1.424368,0.2197848


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.39,"$[-2.83$, $-1.95]$",-10.96,< .001
noun_M,noun_M,-2.45,"$[-2.89$, $-2.01]$",-11.23,< .001
verb_X,verb_X,-2.38,"$[-2.81$, $-1.94]$",-10.88,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.06,"$[-0.22$, $0.34]$",0.51,.867
noun_F_verb_X,noun_F - verb_X,-0.02,"$[-0.30$, $0.26]$",-0.15,.988
noun_M_verb_X,noun_M - verb_X,-0.08,"$[-0.35$, $0.20]$",-0.65,.790


In [232]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 65 of 72

`tmin0==0.11 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.89,1.77,72.50,0.22,.403,.004


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.114491277,0.8107723,0.1251049
noun_M,42,0.021704457,0.8041920,0.1240895
verb_X,42,-0.007786805,0.7901411,0.1219214


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.11,"$[-0.13$, $0.36]$",0.93,.358
noun_M,noun_M,0.02,"$[-0.23$, $0.27]$",0.18,.861
verb_X,verb_X,-0.01,"$[-0.26$, $0.24]$",-0.06,.950


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.09,"$[-0.14$, $0.32]$",0.97,.597
noun_F_verb_X,noun_F - verb_X,0.12,"$[-0.11$, $0.35]$",1.28,.411
noun_M_verb_X,noun_M - verb_X,0.03,"$[-0.20$, $0.26]$",0.31,.949


In [233]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 66 of 72

`tmin0==0.11 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.84,1.96,80.28,0.16,.433,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.6343984,0.9596888,0.1480832
noun_M,42,-0.7420870,0.9349688,0.1442688
verb_X,42,-0.6671857,0.9333663,0.1440215


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.63,"$[-0.93$, $-0.34]$",-4.36,< .001
noun_M,noun_M,-0.74,"$[-1.03$, $-0.45]$",-5.10,< .001
verb_X,verb_X,-0.67,"$[-0.96$, $-0.38]$",-4.59,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.11,"$[-0.10$, $0.31]$",1.27,.419
noun_F_verb_X,noun_F - verb_X,0.03,"$[-0.17$, $0.24]$",0.39,.922
noun_M_verb_X,noun_M - verb_X,-0.07,"$[-0.28$, $0.13]$",-0.88,.654


In [234]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 67 of 72

`tmin0==0.11 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.11,1.73,71.11,0.28,.865,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.3651450,0.8109568,0.1251333
noun_M,42,0.3857896,0.9565765,0.1476030
verb_X,42,0.4160294,0.9127797,0.1408450


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.37,"$[0.09$, $0.64]$",2.64,.010
noun_M,noun_M,0.39,"$[0.11$, $0.66]$",2.79,.007
verb_X,verb_X,0.42,"$[0.14$, $0.69]$",3.01,.004


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.02,"$[-0.28$, $0.23]$",-0.19,.980
noun_F_verb_X,noun_F - verb_X,-0.05,"$[-0.31$, $0.20]$",-0.48,.883
noun_M_verb_X,noun_M - verb_X,-0.03,"$[-0.29$, $0.23]$",-0.28,.957


In [235]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 68 of 72

`tmin0==0.11 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.07,1.85,75.88,0.21,.137,.010


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.3471389,0.7955538,0.1227566
noun_M,42,-0.3049792,0.8314652,0.1282979
verb_X,42,-0.1585840,0.8082495,0.1247156


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.35,"$[-0.60$, $-0.10]$",-2.77,.007
noun_M,noun_M,-0.30,"$[-0.56$, $-0.05]$",-2.43,.018
verb_X,verb_X,-0.16,"$[-0.41$, $0.09]$",-1.27,.210


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.04,"$[-0.27$, $0.19]$",-0.43,.902
noun_F_verb_X,noun_F - verb_X,-0.19,"$[-0.42$, $0.04]$",-1.94,.134
noun_M_verb_X,noun_M - verb_X,-0.15,"$[-0.38$, $0.09]$",-1.51,.294


In [236]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 69 of 72

`tmin0==0.11 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.18,1.88,76.93,0.34,.312,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,3.430646,2.261757,0.3489967
noun_M,42,3.543390,2.185749,0.3372684
verb_X,42,3.355083,2.336922,0.3605950


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,3.43,"$[2.73$, $4.13]$",9.83,< .001
noun_M,noun_M,3.54,"$[2.84$, $4.25]$",10.15,< .001
verb_X,verb_X,3.36,"$[2.65$, $4.06]$",9.61,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.11,"$[-0.41$, $0.18]$",-0.91,.634
noun_F_verb_X,noun_F - verb_X,0.08,"$[-0.22$, $0.37]$",0.61,.814
noun_M_verb_X,noun_M - verb_X,0.19,"$[-0.11$, $0.48]$",1.52,.285


In [237]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 70 of 72

`tmin0==0.11 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.99,1.74,71.48,0.38,.367,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.229821,1.916323,0.2956950
noun_M,42,1.367201,1.833607,0.2829318
verb_X,42,1.201093,1.866273,0.2879721


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.23,"$[0.65$, $1.81]$",4.26,< .001
noun_M,noun_M,1.37,"$[0.79$, $1.95]$",4.73,< .001
verb_X,verb_X,1.20,"$[0.62$, $1.78]$",4.16,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.14,"$[-0.44$, $0.16]$",-1.09,.524
noun_F_verb_X,noun_F - verb_X,0.03,"$[-0.27$, $0.33]$",0.23,.972
noun_M_verb_X,noun_M - verb_X,0.17,"$[-0.14$, $0.47]$",1.32,.391


In [238]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 71 of 72

`tmin0==0.11 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.36,1.75,71.93,0.46,.673,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,4.531062,2.938263,0.4533838
noun_M,42,4.425679,2.625005,0.4050471
verb_X,42,4.521772,2.673249,0.4124913


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,4.53,"$[3.68$, $5.39]$",10.68,< .001
noun_M,noun_M,4.43,"$[3.57$, $5.28]$",10.43,< .001
verb_X,verb_X,4.52,"$[3.67$, $5.38]$",10.66,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.11,"$[-0.22$, $0.43]$",0.76,.726
noun_F_verb_X,noun_F - verb_X,0.01,"$[-0.32$, $0.34]$",0.07,.998
noun_M_verb_X,noun_M - verb_X,-0.10,"$[-0.43$, $0.23]$",-0.70,.766


In [239]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 72 of 72

`tmin0==0.11 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.87,1.74,71.52,0.51,.167,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.890679,2.312613,0.3568439
noun_M,42,2.170637,2.083392,0.3214744
verb_X,42,2.064051,1.989478,0.3069832


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.89,"$[1.23$, $2.55]$",5.74,< .001
noun_M,noun_M,2.17,"$[1.51$, $2.83]$",6.60,< .001
verb_X,verb_X,2.06,"$[1.40$, $2.73]$",6.27,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.28,"$[-0.63$, $0.07]$",-1.91,.141
noun_F_verb_X,noun_F - verb_X,-0.17,"$[-0.52$, $0.18]$",-1.19,.465
noun_M_verb_X,noun_M - verb_X,0.11,"$[-0.24$, $0.46]$",0.73,.747


In [240]:
IDX = 50